# NEURAL MACHINE TRANSLATION - GRU

## Required Module & Config files

In [1]:
import src.RNN_GRU as gruNMT
from src.Tokenizer import Corpus, LangData, dataLoader
from src.utils import load_config, get_device, train_model
from src.Translator import Translator
from torch.nn import CrossEntropyLoss
from torch.optim import NAdam
import evaluate
import numpy as np
from torchinfo import summary

In [2]:
# Loading config file
config = load_config()
# Get device : GPU/MPS Back-End/CPU
device = get_device()
print(f"Using device: {device}")

Using device: mps


## Data Preprocessing

In [3]:
# # TRAIN_DATA
# preprocess_data(config.TRAIN_RAW, config.TRAIN_DATA, config.TRAIN_SOURCE, "english")
# preprocess_data(config.TRAIN_RAW, config.TRAIN_DATA, config.TRAIN_TARGET, "afrikaans")

# # VAL_DATA
# preprocess_data(config.VAL_RAW, config.VAL_DATA, config.VAL_SOURCE, "english")
# preprocess_data(config.VAL_RAW, config.VAL_DATA, config.VAL_TARGET, "afrikaans")

## Load the dataset

In [4]:
# Encoder-Source
english_data = Corpus(f"{config.TRAIN_DATA}/english.txt", "English")
afrikaans_data = Corpus(f"{config.TRAIN_DATA}/afrikaans.txt", "Afrikaans")

## Set Hyperparameters

In [5]:
# Encoder - source
IN_ENCODER = english_data.vocab_size
ENCODER_EMB = 256

# Decoder - target
IN_DECODER = afrikaans_data.vocab_size
OUT_DECODER = afrikaans_data.vocab_size
DECODER_EMB = 256

# Shared
HIDDEN_SIZE = 1024
NUM_LAYERS = 2

LR = 1e-3
BATCH_SIZE = 128

## Set the model

In [10]:
encoder_net = gruNMT.Encoder(IN_ENCODER, ENCODER_EMB, HIDDEN_SIZE, NUM_LAYERS, type="GRU").to(device)
decoder_net = gruNMT.Decoder(IN_DECODER, DECODER_EMB, HIDDEN_SIZE, NUM_LAYERS, type='GRU').to(device)
model = gruNMT.GRU_NMT(encoder_net, decoder_net, OUT_DECODER)

summary(model, depth=4)

Layer (type:depth-idx)                   Param #
GRU_NMT                                  --
├─Encoder: 1-1                           --
│    └─GRU: 2-1                          10,235,904
│    └─Embedding: 2-2                    753,664
├─Decoder: 1-2                           --
│    └─GRU: 2-3                          10,235,904
│    └─Embedding: 2-4                    749,312
│    └─Linear: 2-5                       3,000,175
Total params: 24,974,959
Trainable params: 24,974,959
Non-trainable params: 0

In [12]:
train_data = LangData(english_data, afrikaans_data)
train_loader = dataLoader(train_data, BATCH_SIZE)

pad_idx = afrikaans_data.stoi['<pad>']
criterion = CrossEntropyLoss(ignore_index=0)

optimizer = NAdam(model.parameters(), LR)
translator = Translator(model, english_data, afrikaans_data, device)
metric = evaluate.load("bleu")

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


In [13]:
# Data used for follow-up durring training
mytext = "<sos> given that we represent the target output as $y\in\{0,1\}$ and we have $n$ training points , we can write the negative log likelihood of the parameters as follows: <eos>"
ground = "<sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as: <eos>"

predicted = translator.translate_sentence(mytext)
bleu = metric.compute(predictions=[predicted], references=[ground])
print(f"Pred: {predicted}")
print(f"Refe: {ground}")
for key, val in bleu.items():
	print(f"{key:<20}: {val}")

Pred: <sos> water $x(f_\omega)$ rede rede kak vertaal spyte projeksie boston ""hy ""hy geskrik kyoto (statistiese dubbeltoonmultifrekwensie basketbal eerste noem noem nogal eendimensionele $\frac{\partialj(\mathbf{w})}{\partial\mathbf{w}}$ grootword grootword dan dan eienskap president monsterperiode dan was." monsterperiode dan afgesny sheila
Refe: <sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as: <eos>
bleu                : 0.0
precisions          : [0.17105263157894737, 0.04, 0.013513513513513514, 0.0]
brevity_penalty     : 1.0
length_ratio        : 1.9487179487179487
translation_length  : 76
reference_length    : 39


## Train the data

In [14]:
EPOCHS = 20
params = {
    "model": model,
    "train_loader": train_loader,
    "optimizer": optimizer,
    "criterion": criterion,
    "device": device,
    "epochs": EPOCHS,
    "source_test": mytext,
    "target_test": ground,
	"translator":translator
}

train_model(**params)

Epoch 1/20: 100%|██████████| 20/20 [00:09<00:00,  2.15batch/s, loss=1.785]


Predicted: <sos> die die die die die die die die die die , die die , die , die , die , , die , , die , , , die , , , , die ,
Reference: <sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as: <eos>
BLEU Score: 0.0


Epoch 2/20: 100%|██████████| 20/20 [00:07<00:00,  2.59batch/s, loss=1.547]


Predicted: <sos> die volgende van die volgende van $x[n]$ , en die sein , en die sein , en die sein <eos>
Reference: <sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as: <eos>
BLEU Score: 0.0


Epoch 3/20: 100%|██████████| 20/20 [00:07<00:00,  2.57batch/s, loss=1.341]


...

Predicted: <sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as : <eos>
Reference: <sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as: <eos>
BLEU Score: 0.8764445185661316


Epoch 19/20: 100%|██████████| 20/20 [00:08<00:00,  2.45batch/s, loss=0.250]


Predicted: <sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as : <eos>
Reference: <sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as: <eos>
BLEU Score: 0.8764445185661316


Epoch 20/20: 100%|██████████| 20/20 [00:08<00:00,  2.48batch/s, loss=0.241]

Predicted: <sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as : <eos>
Reference: <sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as: <eos>
BLEU Score: 0.8764445185661316


## EVALUATE

In [16]:
EN_STR = [[' '.join(sent)] for sent in english_data.data_str]
AF_STR = [[' '.join(sent)] for sent in afrikaans_data.data_str]
TRANSLATED = [translator.translate_sentence(sent[0]) for sent in EN_STR]

In [25]:
train_metric = metric.compute(predictions=TRANSLATED, references=AF_STR)
for key, val in train_metric.items():
	print(f"{key:<20}: {val}")

bleu                : 0.9797334473948878
precisions          : [0.9879294585730416, 0.9823358211105668, 0.9771485345255837, 0.9715949247803991]
brevity_penalty     : 1.0
length_ratio        : 1.0066028034854142
translation_length  : 37198
reference_length    : 36954


In [26]:
with open(f"{config.VAL_DATA}/english.txt") as data:
    english_test = data.read().strip().split("\n")
with open(f"{config.VAL_DATA}/afrikaans.txt") as data:
    afrikaans_test = data.read().strip().split("\n")
VAL_AF = [[sent] for sent in afrikaans_test]
VAL_TRANSLATED = [translator.translate_sentence(sent) for sent in english_test]
val_metric = metric.compute(predictions=VAL_TRANSLATED, references=VAL_AF)
for key, val in val_metric.items():
	print(f"{key:<20}: {val}")

bleu                : 0.3957300377631653
precisions          : [0.6580268133379168, 0.48954626334519574, 0.35870002413321533, 0.21346931598382277]
brevity_penalty     : 0.9985572466521466
length_ratio        : 0.9985582864204311
translation_length  : 14545
reference_length    : 14566


In [24]:
with open(f"{config.VAL_DATA}/sun_english.txt") as data:
    sun_english_test = data.read().strip().split("\n")
with open(f"{config.VAL_DATA}/sun_afrikaans.txt") as data:
    sun_afrikaans_test = data.read().strip().split("\n")
SUN_VAL_AF = [[sent] for sent in sun_afrikaans_test]
SUN_VAL_TRANSLATED = [translator.translate_sentence(sent) for sent in sun_english_test]
val_metric = metric.compute(predictions=SUN_VAL_TRANSLATED, references=SUN_VAL_AF)
for key, val in val_metric.items():
	print(f"{key:<20}: {val}")

bleu                : 0.15023826543069776
precisions          : [0.40385591070522575, 0.23191489361702128, 0.13583007266629402, 0.04004711425206125]
brevity_penalty     : 1.0
length_ratio        : 1.0335605663345568
translation_length  : 3942
reference_length    : 3814
